In [ ]:
# ===== Full Transformer HAR Training + Augmentation + Eval =====

import os
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout, LayerNormalization,
    MultiHeadAttention, GlobalAveragePooling1D
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# -----------------------
# Reproducibility
# -----------------------
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# -----------------------
# Paths
# -----------------------
TRAIN_SIGNALS = r"C:\Users\MSI\Desktop\Mitacs Project\Human Activity Recognition\UCI HAR Dataset\train\Inertial Signals\\"
TEST_SIGNALS  = r"C:\Users\MSI\Desktop\Mitacs Project\Human Activity Recognition\UCI HAR Dataset\test\Inertial Signals\\"
TRAIN_LABELS  = r"C:\Users\MSI\Desktop\Mitacs Project\Human Activity Recognition\UCI HAR Dataset\train\y_train.txt"
TEST_LABELS   = r"C:\Users\MSI\Desktop\Mitacs Project\Human Activity Recognition\UCI HAR Dataset\test\y_test.txt"

# Option: evaluate on 50% of the official test set (set to False to keep all)
REDUCE_TEST_BY_HALF = False

# -----------------------
# Data loading utilities
# -----------------------
def load_sensor_matrix(base_path: str, split: str):
    """
    Load 9 Inertial Signals for a given split ('train' or 'test').
    Returns array of shape (samples, 128, 9).
    """
    suffix = f"_{split}.txt"
    names = [
        "body_acc_x", "body_acc_y", "body_acc_z",
        "body_gyro_x", "body_gyro_y", "body_gyro_z",
        "total_acc_x", "total_acc_y", "total_acc_z",
    ]
    mats = []
    for name in names:
        fn = f"{name}{suffix}"
        df = pd.read_csv(os.path.join(base_path, fn), delim_whitespace=True, header=None)
        mats.append(df.values)  # (samples, 128)
    X = np.stack(mats, axis=2)  # (samples, 128, 9)
    return X

def load_labels(path: str):
    return pd.read_csv(path, delim_whitespace=True, header=None).values.flatten()

# -----------------------
# Load train & test
# -----------------------
X_train_full = load_sensor_matrix(TRAIN_SIGNALS, "train")
y_train_full = load_labels(TRAIN_LABELS)

X_test_full  = load_sensor_matrix(TEST_SIGNALS, "test")
y_test_full  = load_labels(TEST_LABELS)

print("Train X:", X_train_full.shape, " Test X:", X_test_full.shape)

# -----------------------
# Label encode (fit on train only)
# -----------------------
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train_full)
y_test_enc  = label_encoder.transform(y_test_full)
num_classes = len(label_encoder.classes_)
print("Classes (encoded):", list(range(num_classes)))

# -----------------------
# Stratified Train/Val split (~20% val ≈ 1470 samples)
# -----------------------
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
train_idx, val_idx = next(sss.split(X_train_full, y_train_enc))

X_train, X_val = X_train_full[train_idx], X_train_full[val_idx]
y_train, y_val = y_train_enc[train_idx], y_train_enc[val_idx]
print("Split -> Train:", X_train.shape, " Val:", X_val.shape)

# -----------------------
# Scale features (fit on TRAIN only, apply to VAL & TEST)
# -----------------------
def scale_by_feature(X_fit, X_list):
    ns, t, f = X_fit.shape
    scaler = StandardScaler()
    X_fit_2d = X_fit.reshape(-1, f)
    scaler.fit(X_fit_2d)
    out = []
    for X in X_list:
        X2 = X.reshape(-1, f)
        Xs = scaler.transform(X2).reshape(X.shape[0], t, f)
        out.append(Xs)
    return out

X_train_scaled, X_val_scaled, X_test_scaled = scale_by_feature(X_train, [X_train, X_val, X_test_full])

# -----------------------
# Data augmentation: jitter classes 3 and 4 (+500 each)
# -----------------------
def jitter(samples, sigma=0.05):
    return samples + np.random.normal(0, sigma, size=samples.shape)

aug_X_list = []
aug_y_list = []
for cls in [0,1,2,3,4]:
    cls_indices = np.where(y_train == cls)[0]
    if len(cls_indices) == 0:
        print(f"Warning: class {cls} not present in training set; skipping augmentation.")
        continue
    chosen = np.random.choice(cls_indices, size=500, replace=True)
    jittered = jitter(X_train_scaled[chosen], sigma=0.05)
    aug_X_list.append(jittered)
    aug_y_list.append(np.full(jittered.shape[0], cls, dtype=y_train.dtype))

if aug_X_list:
    X_aug = np.vstack(aug_X_list)
    y_aug = np.concatenate(aug_y_list)
    X_train_final = np.vstack([X_train_scaled, X_aug])
    y_train_final = np.concatenate([y_train, y_aug])
else:
    X_train_final, y_train_final = X_train_scaled, y_train

print("After augmentation -> Train:", X_train_final.shape, " Val:", X_val_scaled.shape)

# -----------------------
# (Optional) Reduce test by 50% (stratified)
# -----------------------
if REDUCE_TEST_BY_HALF:
    sss_test = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=SEED)
    keep_idx, _ = next(sss_test.split(X_test_scaled, y_test_enc))
    X_test_eval = X_test_scaled[keep_idx]
    y_test_eval = y_test_enc[keep_idx]
else:
    X_test_eval = X_test_scaled
    y_test_eval = y_test_enc

print("Test for evaluation:", X_test_eval.shape)

# -----------------------
# Transformer model
# -----------------------
def build_transformer_model(input_shape, num_classes,
                            embed_dim=64, num_heads=4, ff_dim=128,
                            num_layers=2, dropout_rate=0.2, lr=1e-4):
    inputs = Input(shape=input_shape)

    # Linear projection to embedding space
    x = Dense(embed_dim)(inputs)

    # Stacked Transformer Encoder blocks
    for _ in range(num_layers):
        # Self-attention + residual
        attn = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
        attn = Dropout(dropout_rate)(attn)
        x = LayerNormalization(epsilon=1e-6)(x + attn)

        # Feed-forward + residual
        ffn = Dense(ff_dim, activation="relu")(x)
        ffn = Dense(embed_dim)(ffn)
        ffn = Dropout(dropout_rate)(ffn)
        x = LayerNormalization(epsilon=1e-6)(x + ffn)

    # Sequence pooling
    x = GlobalAveragePooling1D()(x)

    # Head
    x = Dense(128, activation="relu")(x)
    x = Dropout(dropout_rate)(x)

    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=lr),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

input_shape = (X_train_final.shape[1], X_train_final.shape[2])
model = build_transformer_model(input_shape, num_classes,
                                embed_dim=64, num_heads=4, ff_dim=128,
                                num_layers=2, dropout_rate=0.3, lr=1e-4)

model.summary()

# -----------------------
# Training
# -----------------------
callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=12, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", patience=6, factor=0.5, verbose=1)
]

history = model.fit(
    X_train_final, y_train_final,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

# -----------------------
# Evaluation (Validation & Test)
# -----------------------
def evaluate_split(name, X, y_true):
    y_prob = model.predict(X, verbose=0)
    y_pred = np.argmax(y_prob, axis=1)
    acc = accuracy_score(y_true, y_pred)
    print(f"\n=== {name} ===")
    print(f"Accuracy: {acc:.4f}")
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=[str(c) for c in label_encoder.classes_]))
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

evaluate_split("Validation", X_val_scaled, y_val)
evaluate_split("Test", X_test_eval, y_test_eval)

# -----------------------
# (Optional) Save model
# -----------------------
# model.save(r"C:\path\to\save\har_transformer.h5")


C:\Users\MSI\AppData\Local\Temp\ipykernel_19460\1234582450.py:55: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(os.path.join(base_path, fn), delim_whitespace=True, header=None)
C:\Users\MSI\AppData\Local\Temp\ipykernel_19460\1234582450.py:55: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(os.path.join(base_path, fn), delim_whitespace=True, header=None)
C:\Users\MSI\AppData\Local\Temp\ipykernel_19460\1234582450.py:55: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(os.path.join(base_path, fn), delim_whitespace=True, header=None)
C:\Users\MSI\AppData\Local\Temp\ipykernel_19460\1234582450.py:55: FutureWarning: The 'delim_whitespace' keyword in pd.read_c

Train X: (7352, 128, 9)  Test X: (2947, 128, 9)
Classes (encoded): [0, 1, 2, 3, 4, 5]
Split -> Train: (5881, 128, 9)  Val: (1471, 128, 9)
After augmentation -> Train: (6881, 128, 9)  Val: (1471, 128, 9)
Test for evaluation: (2947, 128, 9)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 9)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128, 64)   │        640 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 128, 64)   │     66,368 │ dense[0][0],      │
│ (MultiHeadAttentio… │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128, 64)   │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 128, 64)   │          0 │ dense[0][0],      │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 128, 64)   │        128 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128, 128)  │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 128, 64)   │      8,256 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 128, 64)   │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 128, 64)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 64)   │        128 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 128, 64)   │     66,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 128, 64)   │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 128, 64)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 64)   │        128 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128, 128)  │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128, 64)   │      8,256 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 128, 64)   │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 128, 64)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 64)   │        128 │ add_3[0][0]     

 Total params: 176,134 (688.02 KB)

 Trainable params: 176,134 (688.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 33s 257ms/step - accuracy: 0.4531 - loss: 1.3467 - val_accuracy: 0.7458 - val_loss: 0.6813 - learning_rate: 1.0000e-04
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 27s 245ms/step - accuracy: 0.7556 - loss: 0.6688 - val_accuracy: 0.8668 - val_loss: 0.4562 - learning_rate: 1.0000e-04
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 41s 380ms/step - accuracy: 0.8364 - loss: 0.4795 - val_accuracy: 0.9137 - val_loss: 0.2647 - learning_rate: 1.0000e-04
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 42s 385ms/step - accuracy: 0.9038 - loss: 0.3102 - val_accuracy: 0.9361 - val_loss: 0.1687 - learning_rate: 1.0000e-04
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 21s 195ms/step - accuracy: 0.9198 - loss: 0.2234 - val_accuracy: 0.9483 - val_loss: 0.1374 - learning_rate: 1.0000e-04
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 39s 174ms/step - accuracy: 0.9306 - loss: 0.1883 - val_accuracy: 0.9517 - val_loss: 0.1238 - learning_rate: 1.0000e-04
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━

In [2]:
model.save(r'C:\Users\MSI\Desktop\Mitacs Project\Human Activity Recognition\UCI HAR Dataset\Models\transformer_model.h5')

In [7]:
# -----------------------
# Reduce test to exactly 1473 samples (stratified)
# -----------------------
from sklearn.model_selection import StratifiedShuffleSplit

TARGET_TEST_SIZE = 1473
sss_test = StratifiedShuffleSplit(n_splits=1, test_size=TARGET_TEST_SIZE, random_state=SEED)
keep_idx, _ = next(sss_test.split(X_test_scaled, y_test_enc))

X_test_eval = X_test_scaled[keep_idx]
y_test_eval = y_test_enc[keep_idx]

print("Reduced Test set:", X_test_eval.shape)


Reduced Test set: (1474, 128, 9)


In [ ]:
# -----------------------
# Reduce test to exactly 1473 samples (stratified)
# -----------------------
from sklearn.model_selection import StratifiedShuffleSplit

TARGET_TEST_SIZE = 1473
sss_test = StratifiedShuffleSplit(n_splits=1, test_size=TARGET_TEST_SIZE, random_state=SEED)
keep_idx, _ = next(sss_test.split(X_test_scaled, y_test_enc))

X_test_eval = X_test_scaled[keep_idx]
y_test_eval = y_test_enc[keep_idx]

print("Reduced Test set:", X_test_eval.shape)

def evaluate_split(name, X, y_true):
    y_prob = model.predict(X, verbose=0)
    y_pred = np.argmax(y_prob, axis=1)
    acc = accuracy_score(y_true, y_pred)
    print(f"\n=== {name} ===")
    print(f"Accuracy: {acc:.4f}")
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=[str(c) for c in label_encoder.classes_]))
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

evaluate_split("Validation", X_val_scaled, y_val)
evaluate_split("Test", X_test_eval, y_test_eval)


=== Validation ===
Accuracy: 0.9905
Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       245
           2       1.00      1.00      1.00       215
           3       1.00      1.00      1.00       197
           4       0.96      0.98      0.97       257
           5       0.99      0.96      0.97       275
           6       1.00      1.00      1.00       282

    accuracy                           0.99      1471
   macro avg       0.99      0.99      0.99      1471
weighted avg       0.99      0.99      0.99      1471

Confusion Matrix:
[[245   0   0   0   0   0]
 [  0 215   0   0   0   0]
 [  0   0 197   0   0   0]
 [  0   0   0 253   4   0]
 [  0   0   0  10 265   0]
 [  0   0   0   0   0 282]]

=== Test ===
Accuracy: 0.9369
Classification Report:
              precision    recall  f1-score   support

           1       0.96      0.89      0.92       248
           2       0.95      0.93      0.94       235

In [9]:
# ===== Full Transformer HAR Training + Augmentation + Eval =====

import os
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout, LayerNormalization,
    MultiHeadAttention, GlobalAveragePooling1D
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# -----------------------
# Reproducibility
# -----------------------
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# -----------------------
# Paths
# -----------------------
TRAIN_SIGNALS = r"C:\Users\MSI\Desktop\Mitacs Project\Human Activity Recognition\UCI HAR Dataset\train\Inertial Signals\\"
TEST_SIGNALS  = r"C:\Users\MSI\Desktop\Mitacs Project\Human Activity Recognition\UCI HAR Dataset\test\Inertial Signals\\"
TRAIN_LABELS  = r"C:\Users\MSI\Desktop\Mitacs Project\Human Activity Recognition\UCI HAR Dataset\train\y_train.txt"
TEST_LABELS   = r"C:\Users\MSI\Desktop\Mitacs Project\Human Activity Recognition\UCI HAR Dataset\test\y_test.txt"

# Option: evaluate on 50% of the official test set (set to False to keep all)
REDUCE_TEST_BY_HALF = False

# -----------------------
# Data loading utilities
# -----------------------
def load_sensor_matrix(base_path: str, split: str):
    """
    Load 9 Inertial Signals for a given split ('train' or 'test').
    Returns array of shape (samples, 128, 9).
    """
    suffix = f"_{split}.txt"
    names = [
        "body_acc_x", "body_acc_y", "body_acc_z",
        "body_gyro_x", "body_gyro_y", "body_gyro_z",
        "total_acc_x", "total_acc_y", "total_acc_z",
    ]
    mats = []
    for name in names:
        fn = f"{name}{suffix}"
        df = pd.read_csv(os.path.join(base_path, fn), delim_whitespace=True, header=None)
        mats.append(df.values)  # (samples, 128)
    X = np.stack(mats, axis=2)  # (samples, 128, 9)
    return X

def load_labels(path: str):
    return pd.read_csv(path, delim_whitespace=True, header=None).values.flatten()

# -----------------------
# Load train & test
# -----------------------
X_train_full = load_sensor_matrix(TRAIN_SIGNALS, "train")
y_train_full = load_labels(TRAIN_LABELS)

X_test_full  = load_sensor_matrix(TEST_SIGNALS, "test")
y_test_full  = load_labels(TEST_LABELS)

print("Train X:", X_train_full.shape, " Test X:", X_test_full.shape)

# -----------------------
# Label encode (fit on train only)
# -----------------------
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train_full)
y_test_enc  = label_encoder.transform(y_test_full)
num_classes = len(label_encoder.classes_)
print("Classes (encoded):", list(range(num_classes)))

# -----------------------
# Stratified Train/Val split (~20% val ≈ 1470 samples)
# -----------------------
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
train_idx, val_idx = next(sss.split(X_train_full, y_train_enc))

X_train, X_val = X_train_full[train_idx], X_train_full[val_idx]
y_train, y_val = y_train_enc[train_idx], y_train_enc[val_idx]
print("Split -> Train:", X_train.shape, " Val:", X_val.shape)

# -----------------------
# Scale features (fit on TRAIN only, apply to VAL & TEST)
# -----------------------
def scale_by_feature(X_fit, X_list):
    ns, t, f = X_fit.shape
    scaler = StandardScaler()
    X_fit_2d = X_fit.reshape(-1, f)
    scaler.fit(X_fit_2d)
    out = []
    for X in X_list:
        X2 = X.reshape(-1, f)
        Xs = scaler.transform(X2).reshape(X.shape[0], t, f)
        out.append(Xs)
    return out

X_train_scaled, X_val_scaled, X_test_scaled = scale_by_feature(X_train, [X_train, X_val, X_test_full])

# -----------------------
# Data augmentation: jitter classes 3 and 4 (+500 each)
# -----------------------
def jitter(samples, sigma=0.05):
    return samples + np.random.normal(0, sigma, size=samples.shape)

aug_X_list = []
aug_y_list = []
for cls in [0,1,2,3,4]:
    cls_indices = np.where(y_train == cls)[0]
    if len(cls_indices) == 0:
        print(f"Warning: class {cls} not present in training set; skipping augmentation.")
        continue
    chosen = np.random.choice(cls_indices, size=500, replace=True)
    jittered = jitter(X_train_scaled[chosen], sigma=0.05)
    aug_X_list.append(jittered)
    aug_y_list.append(np.full(jittered.shape[0], cls, dtype=y_train.dtype))

if aug_X_list:
    X_aug = np.vstack(aug_X_list)
    y_aug = np.concatenate(aug_y_list)
    X_train_final = np.vstack([X_train_scaled, X_aug])
    y_train_final = np.concatenate([y_train, y_aug])
else:
    X_train_final, y_train_final = X_train_scaled, y_train

print("After augmentation -> Train:", X_train_final.shape, " Val:", X_val_scaled.shape)

# -----------------------
# (Optional) Reduce test by 50% (stratified)
# -----------------------
if REDUCE_TEST_BY_HALF:
    sss_test = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=SEED)
    keep_idx, _ = next(sss_test.split(X_test_scaled, y_test_enc))
    X_test_eval = X_test_scaled[keep_idx]
    y_test_eval = y_test_enc[keep_idx]
else:
    X_test_eval = X_test_scaled
    y_test_eval = y_test_enc

print("Test for evaluation:", X_test_eval.shape)

# -----------------------
# Define class weights
# -----------------------
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(y_train_final)
# Compute class weights automatically
class_weights_array = compute_class_weight(class_weight='balanced',
                                           classes=classes,
                                           y=y_train_final)
class_weights = {i: w for i, w in enumerate(class_weights_array)}

# Optionally, manually boost the first three classes a bit more
class_weights[0] *= 1.2
class_weights[1] *= 1.2
class_weights[2] *= 1.2

print("Class weights:", class_weights)


# -----------------------
# Transformer model
# -----------------------
def build_transformer_model(input_shape, num_classes,
                            embed_dim=64, num_heads=4, ff_dim=128,
                            num_layers=2, dropout_rate=0.2, lr=1e-4):
    inputs = Input(shape=input_shape)

    # Linear projection to embedding space
    x = Dense(embed_dim)(inputs)

    # Stacked Transformer Encoder blocks
    for _ in range(num_layers):
        # Self-attention + residual
        attn = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
        attn = Dropout(dropout_rate)(attn)
        x = LayerNormalization(epsilon=1e-6)(x + attn)

        # Feed-forward + residual
        ffn = Dense(ff_dim, activation="relu")(x)
        ffn = Dense(embed_dim)(ffn)
        ffn = Dropout(dropout_rate)(ffn)
        x = LayerNormalization(epsilon=1e-6)(x + ffn)

    # Sequence pooling
    x = GlobalAveragePooling1D()(x)

    # Head
    x = Dense(128, activation="relu")(x)
    x = Dropout(dropout_rate)(x)

    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=lr),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

input_shape = (X_train_final.shape[1], X_train_final.shape[2])
model = build_transformer_model(input_shape, num_classes,
                                embed_dim=64, num_heads=4, ff_dim=128,
                                num_layers=2, dropout_rate=0.3, lr=1e-4)

model.summary()

# -----------------------
# Training
# -----------------------
callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=12, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", patience=6, factor=0.5, verbose=1)
]

history = model.fit(
    X_train_final, y_train_final,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

# -----------------------
# Evaluation (Validation & Test)
# -----------------------
def evaluate_split(name, X, y_true):
    y_prob = model.predict(X, verbose=0)
    y_pred = np.argmax(y_prob, axis=1)
    acc = accuracy_score(y_true, y_pred)
    print(f"\n=== {name} ===")
    print(f"Accuracy: {acc:.4f}")
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=[str(c) for c in label_encoder.classes_]))
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

evaluate_split("Validation", X_val_scaled, y_val)
evaluate_split("Test", X_test_eval, y_test_eval)

# -----------------------
# (Optional) Save model
# -----------------------
# model.save(r"C:\path\to\save\har_transformer.h5")


C:\Users\MSI\AppData\Local\Temp\ipykernel_19460\2729547399.py:55: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(os.path.join(base_path, fn), delim_whitespace=True, header=None)
C:\Users\MSI\AppData\Local\Temp\ipykernel_19460\2729547399.py:55: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(os.path.join(base_path, fn), delim_whitespace=True, header=None)
C:\Users\MSI\AppData\Local\Temp\ipykernel_19460\2729547399.py:55: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(os.path.join(base_path, fn), delim_whitespace=True, header=None)
C:\Users\MSI\AppData\Local\Temp\ipykernel_19460\2729547399.py:55: FutureWarning: The 'delim_whitespace' keyword in pd.read_c

Train X: (7352, 128, 9)  Test X: (2947, 128, 9)
Classes (encoded): [0, 1, 2, 3, 4, 5]
Split -> Train: (5881, 128, 9)  Val: (1471, 128, 9)
After augmentation -> Train: (8381, 128, 9)  Val: (1471, 128, 9)
Test for evaluation: (2947, 128, 9)
Class weights: {0: np.float64(1.1318028359216745), 1: np.float64(1.2343151693667156), 2: np.float64(1.300387897595035), 3: np.float64(0.9135600610420754), 4: np.float64(0.8735668125912028), 5: np.float64(1.2416296296296296)}


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 9)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 128, 64)   │        640 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 128, 64)   │     66,368 │ dense_7[0][0],    │
│ (MultiHeadAttentio… │                   │            │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 128, 64)   │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 128, 64)   │          0 │ dense_7[0][0],    │
│                     │                   │            │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 64)   │        128 │ add_4[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 128, 128)  │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 128, 64)   │      8,256 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 128, 64)   │          0 │ dense_9[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 128, 64)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 64)   │        128 │ add_5[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 128, 64)   │     66,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 128, 64)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 128, 64)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 64)   │        128 │ add_6[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 128, 128)  │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 128, 64)   │      8,256 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_13          │ (None, 128, 64)   │          0 │ dense_11[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 128, 64)   │          0 │ layer_normalizat

 Total params: 176,134 (688.02 KB)

 Trainable params: 176,134 (688.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 28s 180ms/step - accuracy: 0.4750 - loss: 1.3043 - val_accuracy: 0.8423 - val_loss: 0.5589 - learning_rate: 1.0000e-04
Epoch 2/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 23s 174ms/step - accuracy: 0.7897 - loss: 0.6200 - val_accuracy: 0.9069 - val_loss: 0.3034 - learning_rate: 1.0000e-04
Epoch 3/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 24s 181ms/step - accuracy: 0.8926 - loss: 0.3537 - val_accuracy: 0.9279 - val_loss: 0.1932 - learning_rate: 1.0000e-04
Epoch 4/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 23s 177ms/step - accuracy: 0.9257 - loss: 0.2308 - val_accuracy: 0.9415 - val_loss: 0.1447 - learning_rate: 1.0000e-04
Epoch 5/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 23s 179ms/step - accuracy: 0.9326 - loss: 0.1768 - val_accuracy: 0.9449 - val_loss: 0.1260 - learning_rate: 1.0000e-04
Epoch 6/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 24s 183ms/step - accuracy: 0.9425 - loss: 0.1582 - val_accuracy: 0.9511 - val_loss: 0.1140 - learning_rate: 1.0000e-04
Epoch 7/100
131/131 ━━━━━━━━━━━━━━━━━━━━

In [10]:
# -----------------------
# Reduce test to exactly 1473 samples (stratified)
# -----------------------
from sklearn.model_selection import StratifiedShuffleSplit

TARGET_TEST_SIZE = 1473
sss_test = StratifiedShuffleSplit(n_splits=1, test_size=TARGET_TEST_SIZE, random_state=SEED)
keep_idx, _ = next(sss_test.split(X_test_scaled, y_test_enc))

X_test_eval = X_test_scaled[keep_idx]
y_test_eval = y_test_enc[keep_idx]

print("Reduced Test set:", X_test_eval.shape)

def evaluate_split(name, X, y_true):
    y_prob = model.predict(X, verbose=0)
    y_pred = np.argmax(y_prob, axis=1)
    acc = accuracy_score(y_true, y_pred)
    print(f"\n=== {name} ===")
    print(f"Accuracy: {acc:.4f}")
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=[str(c) for c in label_encoder.classes_]))
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

evaluate_split("Validation", X_val_scaled, y_val)
evaluate_split("Test", X_test_eval, y_test_eval)

Reduced Test set: (1474, 128, 9)

=== Validation ===
Accuracy: 0.9932
Classification Report:
              precision    recall  f1-score   support

           1       0.99      1.00      1.00       245
           2       1.00      1.00      1.00       215
           3       1.00      0.99      1.00       197
           4       0.98      0.98      0.98       257
           5       0.99      0.98      0.98       275
           6       1.00      1.00      1.00       282

    accuracy                           0.99      1471
   macro avg       0.99      0.99      0.99      1471
weighted avg       0.99      0.99      0.99      1471

Confusion Matrix:
[[245   0   0   0   0   0]
 [  0 215   0   0   0   0]
 [  1   0 196   0   0   0]
 [  0   0   0 253   4   0]
 [  1   0   0   4 270   0]
 [  0   0   0   0   0 282]]

=== Test ===
Accuracy: 0.9050
Classification Report:
              precision    recall  f1-score   support

           1       0.89      0.85      0.87       248
           2       0